# Artificial Neural Network (Classification)

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.1.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
display(dataset.head())

X_df = dataset.iloc[:, 3:-1] # every rows, every columns except last and irrelevant ones
display(X_df.head())

y_df = dataset.iloc[:, -1] # every rows, only last column
display(y_df.head())

X = X_df.values
y = y_df.values

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

### Encoding categorical data

#### Label Encoding the "Gender" column

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X = LabelEncoder() #assign numerical values to categorical values (ex: countries, cities, gender...)
X[:, 2] = labelencoder_X.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding the "Geography" column

In [5]:
# creates column for each categorical value, each row where categorical value applies --> 1, else 0
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))



print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [6]:
X_df2 = pd.DataFrame(X)
X_df2.columns = ['France',
                 'Germany',
                 'Spain', 
                 X_df.columns[0], 
                 X_df.columns[2], 
                 X_df.columns[3], 
                 X_df.columns[4], 
                 X_df.columns[5], 
                 X_df.columns[6], 
                 X_df.columns[7], 
                 X_df.columns[8], 
                 X_df.columns[9]]

display(X_df2.head())

,France,Germany,Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,0,0,619,0,42,2,0,1,1,1,101349
1,0,0,1,608,0,41,1,83807.9,1,0,1,112543
2,1,0,0,502,0,42,8,159661,3,1,0,113932
3,1,0,0,699,0,39,1,0,2,0,0,93826.6
4,0,0,1,850,0,43,2,125511,1,1,1,79084.1


### Splitting the dataset into the Training set and Test set

Training Dataset: The sample of data used to fit the model.
The actual dataset that we use to train the model (weights and biases in the case of Neural Network). 
The model sees and learns from this data.

Test Dataset: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.
provides the gold standard used to evaluate the model.
only used once a model is completely trained
contains carefully sampled data that spans the various classes that the model would face, when used in the real world.

Split dataset into training set (X_train, y_train) and test set (X_test, y_test), 
test_size = proportion of the dataset to include in the test split


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2) # 20% in test set

### Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [10]:
# units=number of neurons(Dataset columns(for input layer)) in first hidden layer
# how many neurons to choose --> experiment --> 6 good number to start
# activation funtion --> used to get the output of node -->to determine the output of neural network like yes or no. 
# ...It maps the resulting values in between 0 to 1 or -1 to 1 etc.
# The rectified linear activation function is a piecewise linear function that will output the input directly if is positive, 
# ...otherwise, it will output zero. 'relu'
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [11]:
# units=number of neurons(Dataset columns(for input layer)) in first hidden layer
# how many neurons to choose --> experiment --> 6 good number to start
# activation funtion --> used to get the output of node -->to determine the output of neural network like yes or no. 
# ...It maps the resulting values in between 0 to 1 or -1 to 1 etc.
# The rectified linear activation function is a piecewise linear function that will output the input directly if is positive, 
# ...otherwise, it will output zero. 'relu'
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [12]:
# output = exited feature, binary output only requires one output neuron
# sigmoid activation function allowed to get predictions + probability that binary outcome = 1 --> customer leaves bank
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
# The ANN needs to be compiled with an optimizer function and a loss function before being trained
# an optimizer function for the network, 
# ....There are several types of optimizers and the choice depends on the nature of the problem
# Loss: used for calculating the losses and errors.
# There are several types and the choice depends on the nature of the problem.
# Metrics: the metric used to measure the accuracy of the model

# adam: --> optimizer performing stochastic gradient descent 
# ...(update weight --> reduce loss error between prediction and real results)
# binary --> loss: 'binary_crossentropy', otherwise 'crossentropy'
# possibility to choose several metrics at the same time --> list

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
# batch learning = more efficient/performant in ANN training
# rather than 1 by 1 prediction comparison --> batch per batch
# classic batch_size value = 32
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 150us/sample - loss: 0.5420 - accuracy: 0.7884
Epoch 2/100
8000/8000 [==============================] - 0s 53us/sample - loss: 0.4679 - accuracy: 0.8070
Epoch 3/100
8000/8000 [==============================] - 0s 50us/sample - loss: 0.4397 - accuracy: 0.8089
Epoch 4/100
8000/8000 [==============================] - 0s 53us/sample - loss: 0.4270 - accuracy: 0.8148
Epoch 5/100
8000/8000 [==============================] - 0s 48us/sample - loss: 0.4200 - accuracy: 0.8177
Epoch 6/100
8000/8000 [==============================] - 0s 52us/sample - loss: 0.4150 - accuracy: 0.8189
Epoch 7/100
8000/8000 [==============================] - 0s 52us/sample - loss: 0.4106 - accuracy: 0.8223
Epoch 8/100
8000/8000 [==============================] - 0s 55us/sample - loss: 0.4066 - accuracy: 0.8248
Epoch 9/100
8000/8000 [==============================] - 0s 50us/sample - loss: 0.4030 - accuracy: 0.8250
Epoch 10/100
8000/8000 

8000/8000 [==============================] - 0s 52us/sample - loss: 0.3323 - accuracy: 0.8668
Epoch 78/100
8000/8000 [==============================] - 0s 51us/sample - loss: 0.3327 - accuracy: 0.8651
Epoch 79/100
8000/8000 [==============================] - 0s 50us/sample - loss: 0.3327 - accuracy: 0.8650
Epoch 80/100
8000/8000 [==============================] - 0s 52us/sample - loss: 0.3323 - accuracy: 0.8654
Epoch 81/100
8000/8000 [==============================] - 0s 52us/sample - loss: 0.3330 - accuracy: 0.8654
Epoch 82/100
8000/8000 [==============================] - 0s 54us/sample - loss: 0.3323 - accuracy: 0.8651
Epoch 83/100
8000/8000 [==============================] - 0s 52us/sample - loss: 0.3326 - accuracy: 0.8674
Epoch 84/100
8000/8000 [==============================] - 0s 49us/sample - loss: 0.3324 - accuracy: 0.8646
Epoch 85/100
8000/8000 [==============================] - 0s 53us/sample - loss: 0.3322 - accuracy: 0.8656
Epoch 86/100
8000/8000 [==========================

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Use our ANN model to predict if the customer with the following informations will leave the bank: 
<br>Geography: France
<br>Credit Score: 600
<br>Gender: Male
<br>Age: 40 years old
<br>Tenure: 3 years
<br>Balance:  USD 60000
<br>Number of Products: 2
<br>Does this customer have a credit card? Yes
<br>Is this customer an Active Member: Yes
<br>Estimated Salary: USD 50000
<br>So, should we say goodbye to that customer?

In [15]:
print(ann.predict(sc_X.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) # as output layer = probability

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5) # predicted binary outcome 0 or 1

y_pred2y_test = pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
y_pred2y_test.columns = ['y_pred', 'y_test']
y_pred2y_test

,y_pred,y_test
0,0,0
1,0,0
2,0,1
3,0,1
4,0,0
...,...,...
1995,0,0
1996,0,0
1997,1,1
1998,0,0


### Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Compute confusion matrix to evaluate the accuracy of a classification.

cm = confusion_matrix(y_test, y_pred)

print(cm)

#accuracy = ((cm[0,0]+cm[1,1])/(cm[0,0]+cm[1,1]+cm[1,0]+cm[1,0]))* 100

accuracy = accuracy_score(y_test, y_pred) *100

# \n = begin new paragraph

print('\nThe Model is {0}% accurate'.format(round(accuracy, 2)))

[[1527   55]
 [ 231  187]]

The Model is 85.7% accurate


An artificial neural network (ANN) is the piece of a computing system designed to simulate the way the human brain analyzes and processes information. It is the foundation of artificial intelligence (AI) and solves problems that would prove impossible or difficult by human or statistical standards. ANNs have self-learning capabilities that enable them to produce better results as more data becomes available.